In [52]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, count, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [53]:
spark = SparkSession.builder.appName("Diabetes").getOrCreate()

input_data = "C:/Users/yagoc/Documents/Linguagem Java/Programação para Ciência de Dados/Projeto_Diabetes/diabetes.csv"

diabetes_df = spark.read.csv(input_data,header=True, inferSchema=True)

diabetes_df.show(20)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


In [54]:
#contar valores nulos
diabetes_df.select([count(when(col(c).isNull(), c)).alias(c) for c in diabetes_df.columns]).show()
#RESULTADO: Não há valores missing

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [55]:
#Descreve os dados
diabetes_df.describe().show() #ou diabetes_df.summary().show() para quartis

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|  count|               768|              768|               768|               768|               768|               768|                     768|               768|               768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|      0.4718763020833327|33.240885416666664|0.3489583333333333|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.95

In [56]:
#Dividir em dados de treino e teste
treinamento_df, teste_df = diabetes_df.randomSplit([.7,.3], seed=42)

In [57]:
#Mostrar tipos de dados das colunas
diabetes_df.printSchema()
#RESULTADO: Nãos erá necessário converter colunas de string para numérica

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [58]:
#Juntar os atributos em uma única coluna
vector_assembler = VectorAssembler(
                                    inputCols=diabetes_df.columns,
                                    outputCol="features"
                                  )

In [59]:
#Regressão Logística
lr = LogisticRegression(
                        featuresCol="features",
                        labelCol="Outcome"
                       )

In [60]:
#Criar o pipeline
pipeline = Pipeline(stages=[vector_assembler, lr])

In [61]:
#Treinar o modelo
pipeline_model = pipeline.fit(treinamento_df)

In [62]:
#Testar o modelo
predict_df = pipeline_model.transform(teste_df)

In [ ]:
#Resultados
predict_df.select("Outcome", "prediction").show(25)

In [64]:
#Acurácia
evaluator = MulticlassClassificationEvaluator(
                                              predictionCol="prediction",
                                              labelCol="Outcome",
                                              metricName="accuracy"
                                             )

acuracia = evaluator.evaluate(predict_df)

print("Acurácia da Classificação: " + str(acuracia) + "\n")

Acurácia da Classificação: 1.0

